# 卷积神经网络

## 二维互相关运算

In [1]:
import torch
from  torch import nn

def corr2D(X,kernel):
    h,w = kernel.shape
    Y = torch.zeros(size=(X.shape[0]-h+1,X.shape[1]-w+1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i,j] = (X[i:i+h,j:j+w]*kernel).sum()
    return Y

## 卷积层

In [2]:
class Conv2D(nn.Module):
    def __init__(self,kernel_size):
        super().__init__()
        self.weight = torch.rand(kernel_size)
        self.bias = torch.zeros(1)

    def forward(self,X):
        return corr2D(X,self.weight)+self.bias

## 训练过程
卷积神经网络训练的权值就是卷积核，比如本例中通过类似Sobel算子的卷积核进行滤波后得出原图像上的边界，而训练就是尝试训练出一个卷积核类似Sobel算子，同样能识别边界

In [3]:
X = torch.ones((6, 8))
X[:,2:6] = 0
kernel = torch.tensor([[1,-1]])
Y = corr2D(X,kernel)

conv2d = nn.Conv2d(1,1,kernel_size=(1,2),bias=False)
X = X.reshape(1,1,6,8) # 批量大小、通道、高度、宽度
Y = Y.reshape(1,1,6,7)
lr = 0.03

for i in range(10):
    y_hat = conv2d(X)
    loss = (y_hat-Y)**2
    conv2d.zero_grad()
    loss.sum().backward()
    conv2d.weight.data[:] -= lr*conv2d.weight.grad
    print(f'epoch{i} loss:{loss.sum()}')

print(conv2d.weight.data)

epoch0 loss:15.000081062316895
epoch1 loss:6.1703267097473145
epoch2 loss:2.5441935062408447
epoch3 loss:1.0528714656829834
epoch4 loss:0.4381486773490906
epoch5 loss:0.18387696146965027
epoch6 loss:0.07813920080661774
epoch7 loss:0.033812664449214935
epoch8 loss:0.015006041154265404
epoch9 loss:0.006886557210236788
tensor([[[[ 0.9825, -0.9918]]]])


## 填充与步幅

In [4]:
import torch
from torch import nn

def comp_conv2d(conv2d,X):
    h,w = X.shape
    X = X.reshape(1,1,h,w)
    Y = conv2d(X)
    return Y.reshape(Y.shape[2:]) #批量大小、通道、高度、宽度，reshape成一个高度*宽度大小的张量
conv2d = nn.Conv2d(1,1,kernel_size=3,padding=1)
X = torch.rand(size=(8,8))
print(comp_conv2d(conv2d,X).shape)

conv2d_ = nn.Conv2d(1,1,kernel_size=3,padding=1,stride=2)
print(comp_conv2d(conv2d_,X).shape)

torch.Size([8, 8])
torch.Size([4, 4])


## 多通道

### 多通道输入

In [5]:
import torch
import My_utils

def corr2d_multi_in(X,K):
    return sum(My_utils.corr2D(x,k) for x,k in zip(X,K))

X = torch.tensor([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]],
                  [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])
K = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])

corr2d_multi_in(X,K)


tensor([[ 56.,  72.],
        [104., 120.]])

### 多通道输出

In [6]:
def corr2d_multi_in_out(X,K):
    return torch.stack([corr2d_multi_in(X,k) for k in K],0)

K = torch.stack([K,K+1,K+2],0)
corr2d_multi_in_out(X,K)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

## 池化层

In [7]:
import torch
import My_utils
from torch import nn
def pool2d(pool_size,X,mode='max'):
    p_h,p_w = pool_size
    Y = torch.zeros((X.shape[0]-p_h+1,X.shape[1]-p_w+1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            if mode == 'max':
                Y[i,j] = X[i:i+p_h,j:j+p_w].max()
            elif mode == 'avg':
                Y[i,j] = X[i:i+p_h,j:j+p_w].mean()
    return Y

X = torch.tensor([[1,2,3],[4,5,6],[7,8,9]])
print(pool2d((2,2),X))
print(pool2d((2,2),X.type(torch.float),mode='avg'))

tensor([[5., 6.],
        [8., 9.]])
tensor([[3., 4.],
        [6., 7.]])


In [8]:
X = torch.arange(16,dtype=float).reshape((1,1,4,4))
pooling2d = nn.MaxPool2d(2) # pytorch中，池化窗口的大小和步幅相同 -> 不同的池化窗口没有重叠
print(pooling2d(X))

X = torch.cat((X,X+1),1)
pooling2d = nn.MaxPool2d(2,stride=1)
print(pooling2d(X))

tensor([[[[ 5.,  7.],
          [13., 15.]]]], dtype=torch.float64)
tensor([[[[ 5.,  6.,  7.],
          [ 9., 10., 11.],
          [13., 14., 15.]],

         [[ 6.,  7.,  8.],
          [10., 11., 12.],
          [14., 15., 16.]]]], dtype=torch.float64)
